In [325]:
data = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"
!wget $data -O course_lead_scoring.csv

--2025-12-21 16:47:56--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.02s   

2025-12-21 16:47:56 (3.86 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [326]:
import pandas as pd
import numpy as np
import sklearn as sk

df = pd.read_csv("course_lead_scoring.csv")

In [327]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [328]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [329]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [330]:
df.nunique()

lead_source                    5
industry                       7
number_of_courses_viewed      10
annual_income               1267
employment_status              4
location                       7
interaction_count             12
lead_score                   101
converted                      2
dtype: int64

# 1

In [331]:
# most frequent category in 'industry' column
df.industry.mode()[0]

'retail'

# 2

In [332]:
# create the correlation matrix for all numerical features
numerical = [col for col in df.columns if col not in categorical]
numerical

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [333]:
corr_matrix = df[numerical].corr()
corr_matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.031551,-0.023565,-0.004879,0.435914
annual_income,0.031551,1.000000,0.048618,0.005334,0.078256
interaction_count,-0.023565,0.048618,1.000000,0.009888,0.374573
lead_score,-0.004879,0.005334,0.009888,1.000000,0.193673
converted,0.435914,0.078256,0.374573,0.193673,1.000000


In [334]:
# the maximum absolute correlation value (the absolute value of the correlation coefficient)
# (excluding the correlation of the feature with itself)
corr_matrix.abs().unstack().sort_values(ascending=False).drop_duplicates()

number_of_courses_viewed  number_of_courses_viewed    1.000000
converted                 number_of_courses_viewed    0.435914
interaction_count         converted                   0.374573
lead_score                converted                   0.193673
converted                 annual_income               0.078256
interaction_count         annual_income               0.048618
number_of_courses_viewed  annual_income               0.031551
                          interaction_count           0.023565
interaction_count         lead_score                  0.009888
lead_score                annual_income               0.005334
number_of_courses_viewed  lead_score                  0.004879
dtype: float64

In [335]:
data_numeric = df.copy()
data_numeric = data_numeric.drop(
    ['lead_source', 'industry', 'employment_status', 'location', 'converted'],
    axis=1
)
data_numeric.corr()



,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.031551,-0.023565,-0.004879
annual_income,0.031551,1.000000,0.048618,0.005334
interaction_count,-0.023565,0.048618,1.000000,0.009888
lead_score,-0.004879,0.005334,0.009888,1.000000


Time to split the data...

In [336]:
df_full_train, df_test = sk.model_selection.train_test_split(df, test_size=0.2, random_state=42)
print(len(df_full_train), len(df_test))

1169 293


In [337]:
df_train, df_val = sk.model_selection.train_test_split(df_full_train, test_size=0.25, random_state=42)
print(len(df_train), len(df_val))

876 293


In [338]:
df_full_train = df_full_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

y_full_train = df_full_train.converted.values
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_full_train['converted']
del df_train['converted']
del df_val['converted']
del df_test['converted']

In [339]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']

In [340]:
for col in categorical:
    print(f"Column: {col}")
    print(df[col].value_counts())
    print()
    print()

Column: lead_source
lead_source
organic_search    282
social_media      278
paid_ads          264
referral          260
events            250
Name: count, dtype: int64


Column: industry
industry
retail           203
finance          200
other            198
education        187
healthcare       187
technology       179
manufacturing    174
Name: count, dtype: int64


Column: employment_status
employment_status
self_employed    352
student          348
unemployed       334
employed         328
Name: count, dtype: int64


Column: location
location
north_america    225
europe           216
middle_east      198
asia             195
south_america    192
africa           188
australia        185
Name: count, dtype: int64




In [341]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [342]:
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,NaN,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NaN,5,71016.0,self_employed,north_america,0,0.25,1


In [343]:
# create the correlation matrix for all numerical features
numerical = [col for col in df.columns if col not in categorical + ['converted']]
numerical

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [344]:
corr_matrix = df[numerical].corr()
corr_matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.031551,-0.023565,-0.004879
annual_income,0.031551,1.000000,0.048618,0.005334
interaction_count,-0.023565,0.048618,1.000000,0.009888
lead_score,-0.004879,0.005334,0.009888,1.000000


In [345]:
# the maximum absolute correlation value (the absolute value of the correlation coefficient)
# (excluding the correlation of the feature with itself)
corr_matrix.abs().unstack().sort_values(ascending=False).drop_duplicates()

number_of_courses_viewed  number_of_courses_viewed    1.000000
annual_income             interaction_count           0.048618
                          number_of_courses_viewed    0.031551
number_of_courses_viewed  interaction_count           0.023565
interaction_count         lead_score                  0.009888
lead_score                annual_income               0.005334
number_of_courses_viewed  lead_score                  0.004879
dtype: float64

In [346]:
data_numeric = df.copy()
data_numeric = data_numeric.drop(
    ['lead_source', 'industry', 'employment_status', 'location', 'converted'],
    axis=1
)
data_numeric.corr()



,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.031551,-0.023565,-0.004879
annual_income,0.031551,1.000000,0.048618,0.005334
interaction_count,-0.023565,0.048618,1.000000,0.009888
lead_score,-0.004879,0.005334,0.009888,1.000000


Time to split the data...

In [347]:
df_full_train, df_test = sk.model_selection.train_test_split(df, test_size=0.2, random_state=42)
print(len(df_full_train), len(df_test))

1169 293


In [348]:
df_train, df_val = sk.model_selection.train_test_split(df_full_train, test_size=0.25, random_state=42)
print(len(df_train), len(df_val))

876 293


In [349]:
df_full_train = df_full_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

y_full_train = df_full_train.converted.values
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

df_train = df_train.drop('converted', axis=1)
df_val = df_val.drop('converted', axis=1)
df_test = df_test.drop('converted', axis=1)

assert 'converted' not in df_train.columns
assert 'converted' not in df_val.columns
assert 'converted' not in df_test.columns

# 3

In [350]:
def calculate_mi(series):
    return sk.metrics.mutual_info_score(series, y_train)

cat = ['lead_source', 'industry', 'employment_status', 'location']
numeric = ['number_of_courses_viewed', 'annual_income', 'interaction_count',
       'lead_score']
df_mi = df_train[categorical].fillna('NA').apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')
df_mi

,MI
lead_source,0.035396
employment_status,0.012938
industry,0.011575
location,0.004464


# 4

In [351]:
df_train.isnull().sum()

lead_source                  89
industry                     78
number_of_courses_viewed      0
annual_income               110
employment_status            66
location                     38
interaction_count             0
lead_score                    0
dtype: int64

In [357]:

cat = ['lead_source', 'industry', 'employment_status', 'location']
numeric = ['number_of_courses_viewed', 'annual_income', 'interaction_count',
       'lead_score']

df_train[cat] = df_train[categorical].fillna('NA')
df_train[numeric] = df_train[numeric].fillna(0)
df_train.isnull().sum()


lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
dtype: int64

In [358]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=False)
train_dicts = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [365]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# the reference solution doesn't use liblinear even though the exercise demands it

/home/teo/workspace/machine-learning-zoomcamp-homework/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [366]:

df_val[cat] = df_val[cat].fillna('NA')
df_val[numeric] = df_val[numeric].fillna(0)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

y_pred = model.predict(X_val)

sk.metrics.accuracy_score(y_val, y_pred)

0.8532423208191127

# 5

In [368]:
for drop_feature in ['lead_score', 'industry', 'employment_status']:
    df_train_temp = df_train.drop(columns=[drop_feature])
    df_val_temp = df_val.drop(columns=[drop_feature])
    
    dv_temp = DictVectorizer(sparse=False)
    train_dicts_temp = df_train_temp.to_dict(orient='records')
    X_train_temp = dv_temp.fit_transform(train_dicts_temp)
    
    model_temp = LogisticRegression(C=1.0, max_iter=1000, random_state=42)
    model_temp.fit(X_train_temp, y_train)
    
    val_dict_temp = df_val_temp.to_dict(orient='records')
    X_val_temp = dv_temp.transform(val_dict_temp)
    
    y_pred_temp = model_temp.predict(X_val_temp)
    
    accuracy = sk.metrics.accuracy_score(y_val, y_pred_temp)
    print(f"Dropping {drop_feature}, Accuracy: {accuracy}")
    print(f"Accuracy difference: {sk.metrics.accuracy_score(y_val, y_pred) - accuracy}")

/home/teo/workspace/machine-learning-zoomcamp-homework/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/teo/workspace/machine-learning-zoomcamp-homework/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale t

Dropping lead_score, Accuracy: 0.8191126279863481
Accuracy difference: 0.03412969283276457
Dropping industry, Accuracy: 0.8395904436860068
Accuracy difference: 0.013651877133105894
Dropping employment_status, Accuracy: 0.8327645051194539
Accuracy difference: 0.020477815699658786


/home/teo/workspace/machine-learning-zoomcamp-homework/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# 6

In [369]:
# regulariser logistic regression
for c in [0.01, 0.1, 1, 10, 100]:
    model = LogisticRegression(C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = sk.metrics.accuracy_score(y_val, y_pred)
    print(f"C={c}, Accuracy: {accuracy}")

C=0.01, Accuracy: 0.8122866894197952
C=0.1, Accuracy: 0.8430034129692833
C=1, Accuracy: 0.8532423208191127
C=10, Accuracy: 0.8532423208191127
C=100, Accuracy: 0.8532423208191127


/home/teo/workspace/machine-learning-zoomcamp-homework/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


The reference solution says 1 like me, but the question solutions page says 0.01